Importando bibliotecas

In [1]:
from xml.dom import minidom
import urllib
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

Elsevier API Key

In [2]:
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxx"

Preparando a consulta ao Science Direct. Como a API limita o tamanho da consulta, serão feitas diversas consulta cada uma retornando 200 resultados. A consulta "aff(instituição)" retorna os artigos cujo um dos autores estão afiliados a instituição entre parênteses.

O objetivo dessa etapa é recuperar o DOI de todos os artigos cujo autor pertence a uma determinada instituição. O DOI é o Identificador Digital do Objeto usado pela Science Direct.

In [3]:
def get_DOIs (query, finish):
    DOIs = []                  # Lista para armazenar os DOI
    #query =                   # Consulta da instituição

    start = 1
    step =  200
    #finish = 1500             # Número máximo de artigos buscados

    while start < finish:

        url = "https://api.elsevier.com/content/search/scidir?query=" + query + "&count=" + str(step) + "&start=" + str(start) + "&APIKey=" + api_key
        f = requests.get(url).text
        f = json.loads(f)
        try:
            n_entries = len(f["search-results"]["entry"])
            for entry in f["search-results"]["entry"]:
                try:
                    DOIs.append(entry['dc:identifier'])
                except:
                    pass
        except:
            pass
        
        start = start + step
    
    return(DOIs)

Recuperando dados dos artigos

In [4]:
def get_article(article):
    
    article = BeautifulSoup(article, "lxml")

    try:
        DOI = article.find("dc:identifier").get_text()[4:]
    except:
        DOI = ""
        
    try:
        Title = article.find("dc:title").get_text()
    except:
        Title = ""
    
    try:
        Date = article.find("prism:coverdate").get_text()    
    except:
        Date = ""
    
    try:
        Abstract = article.find("dc:description").get_text()
    except:
        Abstract = ""
    
    Keywords = []
    Creators = []
    try:
        coredata = article.find("coredata")
        for data in coredata.descendants:
            try:
                if data.name == "dcterms:subject":
                    Keywords.append(data.get_text())
            except:
                pass
            try:
                if data.name == "dc:creator":
                    Creators.append(data.get_text())
            except:
                pass
    except:
        pass
            
    row = pd.Series([DOI, Title, Date, Abstract, Keywords, Creators],
                    index=["DOI", "Title", "Date", "Abstract", "Keywords", "Creator"])
            
    return(row)

In [5]:
def get_abstracts(query, query_count, name_file):
    articles = { "DOI":[], "Title": [], "Abstract": []}
    articles = pd.DataFrame(data=articles)

    #query = "aff(Petrobras)"
                    # "date(2018)&sortBy=relevance"
                    # "aff(Petrobras)"

    DOIs = get_DOIs (query, query_count)

    n = 0
    for DOI in DOIs:
        f = requests.get("https://api.elsevier.com/content/article/doi/" + DOI[4:] + "?APIKey=" + api_key)
        f = f.text
        articles = articles.append(get_article(f), ignore_index=True)
        n = n + 1
        if n % 100 == 0:
            print ('Artigos: ', n)

    articles.to_json(name_file + ".json")
    return(articles)

Extrair artigos das Majors

In [36]:
Exxon = get_abstracts("aff(Exxon)", 2400, "Elsevier_abstracts - Exxon")
Exxon

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000
Artigos:  1100
Artigos:  1200
Artigos:  1300
Artigos:  1400
Artigos:  1500
Artigos:  1600
Artigos:  1700
Artigos:  1800
Artigos:  1900
Artigos:  2000
Artigos:  2100
Artigos:  2200
Artigos:  2300


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Publisher Summary\n \...,10.1016/B978-188420760-0.50016-8,Plastomer Modified Polypropylene Films for Me...,"[Portnoy, R.C.]",1998-12-31,[]
1,\n Publisher Summary\n \...,10.1016/B978-188420760-0.50010-7,Understanding Environmental Stress Cracking i...,"[Lustiger, Arnold]",1998-12-31,[]
2,,10.1016/B0-08-043152-6/00439-3,"Elastomers, α-Olefin","[Ver Strate, G.]",2001-12-31,[]
3,,10.1016/B0-08-043152-6/01269-9,Polydienes,"[Fetters, L.J.]",2001-12-31,[]
4,\n Abstract\n \n ...,10.1016/S0167-8922(99)80042-X,The importance of the stribeck curve in the mi...,"[Bovington, C., Korcek, S., Sorab, J.]",1999-12-31,[]
5,\n I. ABSTRACT\n Injecti...,10.1016/B978-0-12-641750-0.50017-7,SOME PHYSICOCHEMICAL ASPECTS OF MICROEMULSION ...,"[Reed, Ronald L., Healy, Robert N.]",1977-12-31,[]
6,,10.1016/B978-0-08-034819-3.50016-9,RECRUITMENT TO THE CHEMICAL ENGINEERING PROFES...,"[Hackitt, J.E.]",1987-12-31,[]
7,,10.1016/B978-0-08-037018-7.50005-2,CHAPTER 1 Is 3-D Wave-Equation Modeling Feasib...,"[BAKER, L.J.]",1989-12-31,[]
8,\n We have recently prepared a poly...,10.1016/B978-0-12-432250-9.50021-6,GRAFT CURING WITH A MODIFIED BUTYL RUBBER,"[Baldwin, F.P., Gardner, I.J.]",1977-12-31,[]
9,\n Abstract\n \n ...,10.1016/0161-8938(93)90008-E,An applied general equilibrium model of intern...,"[York, Harold L.]",1993-12-31,[]


In [38]:
Shell = get_abstracts("aff(Shell)", 4700, "Elsevier_abstracts - Shell")
Shell

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000
Artigos:  1100
Artigos:  1200
Artigos:  1300
Artigos:  1400
Artigos:  1500
Artigos:  1600
Artigos:  1700
Artigos:  1800
Artigos:  1900
Artigos:  2000
Artigos:  2100
Artigos:  2200
Artigos:  2300
Artigos:  2400
Artigos:  2500
Artigos:  2600
Artigos:  2700
Artigos:  2800
Artigos:  2900
Artigos:  3000
Artigos:  3100
Artigos:  3200
Artigos:  3300
Artigos:  3400
Artigos:  3500
Artigos:  3600
Artigos:  3700
Artigos:  3800
Artigos:  3900
Artigos:  4000
Artigos:  4100
Artigos:  4200
Artigos:  4300
Artigos:  4400
Artigos:  4500
Artigos:  4600


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/S1363-4127(99)80084-3,Designing a practical Public Key Infrastructur...,"[Mansfield, Nick]",1999-12-31,[]
1,,10.1016/S0269-915X(09)80645-9,Bioremediation of earth,"[Morgan, Philip, Watkinson, Robert]",1993-05-31,[]
2,\n Abstract\n \n ...,10.1016/B978-0-12-651250-2.50017-X,10 A General Description of the Breaking Proce...,"[SCOTT, J.A.N.]",1976-12-31,[]
3,Unknown,10.1016/S0024-6301(78)80019-3,\n \n \n ...,"[Jefferson, Michael]",1978-06-30,[]
4,\n \n KEYWORDS:\n ...,10.1016/B978-0-08-037115-3.50012-2,Strategic decision making and simulation in Sh...,"[Kalff, D.J. A.]",1989-12-31,[]
5,Unknown,10.1016/S0022-0728(77)80435-X,\n \n \n ...,"[Williams, K.R.]",1977-05-10,[]
6,Unknown,10.1016/S0022-0728(73)80403-6,\n \n Electroch...,"[Williams, K.R.]",1973-03-10,[]
7,,10.1016/0301-4215(91)90080-8,Interdependence on the oil bridge — Risks and ...,"[Tempest, Paul]",1991-02-28,[]
8,,10.1016/0301-4215(90)90154-V,An historical perspective on oil Oil in the wo...,"[Tempest, Paul]",1990-03-31,[]
9,,10.1016/0264-8172(89)90035-4,Petroleum geology and geophysics research in a...,"[James, K.H.]",1989-11-30,[]


In [39]:
Chevron = get_abstracts("aff(Chevron)", 1200, "Elsevier_abstracts - Chevron")
Chevron

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000
Artigos:  1100


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.jngse.2011.07.013,Application of case-based reasoning for well f...,"[Popa, Andrei, Wood, William]",2011-12-31,"[Case-based reasoning, Hydraulic fracturing, A..."
1,,10.1016/0737-6782(92)90079-R,Successful product development: Management pra...,"[Panton, Chris J.]",1992-03-31,[]
2,,10.1016/0737-6782(95)90177-9,Business blindspots: Replacing your company's ...,"[Panton, Chris]",1995-11-30,[]
3,\n Abstract\n \n ...,10.1016/S1387-1811(98)00106-1,Guest/host relationships in zeolite synthesis:...,"[Nakagawa, Y, Lee, G.S, Harris, T.V, Yuen, L.T...",1998-06-17,"[Structure-directing agent, Substituted piperi..."
4,,10.1016/B978-1-4832-3245-4.50019-4,CHAPTER F The Numerical Analysis and Kinetic I...,"[Goodrich, F.C.]",1967-12-31,[]
5,,10.1016/B978-1-4832-3245-4.50020-0,CHAPTER G Additional Methods of Fractionation,"[Cantow, Manfred J.R.]",1967-12-31,[]
6,,10.1016/B978-1-4832-3245-4.50009-1,CHAPTER B.3 Chromatographic Fractionation,"[Porter, Roger S., Johnson, Julian F.]",1967-12-31,[]
7,\n KEYWORDS\n Incorporat...,10.1016/B978-0-08-025396-1.50012-9,THE ENHANCEMENT OF PLATINUM'S CATALYTIC ACTIVI...,"[Kluksdahl, Harris E.]",1983-12-31,[]
8,\n Abstract\n \n ...,10.1016/j.coldregions.2009.04.002,State of the art of ice bearing capacity and i...,"[Masterson, D.M.]",2009-09-30,"[Ice, Ice engineering, Sea ice, Ice bearing ca..."
9,\n \n The determinati...,10.1016/B978-0-12-628750-9.50031-0,THE DETERMINATION OF ADSORPTION ENERGY DISTRIB...,"[Merz, P.H.]",1980-12-31,[]


In [40]:
Total = get_abstracts("aff(Total)", 1600, "Elsevier_abstracts - Total")
Total

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000
Artigos:  1100
Artigos:  1200
Artigos:  1300
Artigos:  1400
Artigos:  1500


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/B978-0-08-011577-1.50025-9,COMPLEXES METALLIQUES DANS LES BRUTS: RELATION...,"[ARICH, G., COSTANTINIDES, G.]",1966-12-31,[]
1,,10.1016/S1361-3723(00)06012-7,Protect your laptop against theft,"[Burton, Terry]",2000-06-01,[]
2,\n \n Keywords Fluid ...,10.1016/B978-0-444-53042-4.00024-8,24 Fluid flow in sedimentary basins including ...,"[Wendebourg, Johannes]",2012-12-31,[]
3,\n \n Since its...,10.3182/20090610-3-IT-4004.00040,Dependability & safety modeling and calculatio...,"[Signoret, Jean-Pierre]",2009-06-30,"[Petri Net, Monte Carlo, Stochastic process, R..."
4,\n Abstract\n \n ...,10.1016/j.egypro.2017.07.151,Fired equipment safety in the oil & gas indust...,"[Dugué, Jacques]",2017-08-31,"[Fired equipment safety, risk analysis, HAZOP,..."
5,,10.1016/j.jvsv.2017.12.018,Etiology of Iliac Vein Stent Thrombosis,"[Hingorani, Anil, Marks, Natalie, Ascher, Enrico]",2018-03-31,[]
6,,10.1016/j.jamda.2017.12.066,Reducing Falls in the Frail Elderly,"[Hockman-McDowell, Nora, Hockman-McDowell, N.M...",2018-03-31,[]
7,\n \n This arti...,10.1016/j.proeng.2012.08.345,REMOVED: Seawater Treatment using UF and NF Me...,"[Jacob, M., Lorain, O., Espenan, J.M., Lesage,...",2012-12-31,[]
8,,10.1016/j.gca.2006.06.1486,Large scale introduction of compaction water e...,"[Walgenwitz, F., Jacquemet, N.]",2006-09-30,[]
9,,10.1016/j.clae.2012.10.057,Getting started: A simple guide to fitting the...,"[Rose, Paul]",2012-12-01,[]


In [41]:
Eni = get_abstracts("aff(Eni)", 800, "Elsevier_abstracts - Eni")
Eni

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/B0-12-176480-X/00247-3,Modeling Energy Supply and Demand: A Compariso...,"[Lanza, Alessandro, Bosello, Francesco]",2004-12-31,[]
1,\n Abstract\n \n ...,10.1016/j.ecolecon.2005.03.031,Reassessing the environmental Kuznets curve fo...,"[Galeotti, Marzio, Lanza, Alessandro, Pauli, F...",2006-04-15,"[Environment, Growth, CO2 emissions, Panel data]"
2,,10.1016/0304-5102(87)85033-2,Hydrolysis of sugars by microbial cells entrap...,"[Marconi, W., Pansolli, P., Giovenco, S.]",1987-09-30,[]
3,\n Abstract\n \n ...,10.1016/B978-0-12-804436-0.00003-5,Chapter 3 Renewable Energy in the Southern and...,"[Tagliapietra, Simone]",2016-12-31,"[renewables, solar, wind, electricity, sustain..."
4,\n Abstract\n \n ...,10.1016/B978-0-12-804436-0.00006-0,Chapter 6 Toward a New Euro-Mediterranean Ener...,"[Hafner, Manfred, Tagliapietra, Simone]",2016-12-31,"[Euro-Med energy cooperation, hydrocarbons, en..."
5,,10.1016/j.jbiotec.2010.09.859,Revival and developments of the ABE process fo...,"[Serbolisca, L., Cardaci, A., Bianchi, D., de ...",2010-11-30,[]
6,\n Abstract\n \n ...,10.1016/j.fuproc.2011.07.011,Methyldibenzothiophene isomer ratio in crude o...,"[Chiaberge, Stefano, Fiorani, Tiziana, Cesti, ...",2011-11-30,"[Crude oil, Geochemical markers, Methyldibenzo..."
7,\n Abstract\n \n ...,10.1016/j.vibspec.2017.06.001,Fingerprinting of [60]fullerene derivatives su...,"[Barbieri, Riccardo, Abbondanza, Luigi, Fiocca...",2017-09-30,"[Cycloaddition, DFT calculations, [60]Fulleren..."
8,\n Abstract\n \n ...,10.1016/j.cattod.2009.01.006,Gas to liquids technologies for natural gas re...,"[Perego, C., Bortolo, R., Zennaro, R.]",2009-04-15,"[Fischer–Tropsch, Gas-to-liquid, Catalyst design]"
9,\n \n Alkylatio...,10.1016/S1878-7959(09)00102-9,Chapter 2 Advances in Aromatics Processing Usi...,"[Perego, C., Pollesel, P.]",2010-12-31,[]


In [42]:
Repsol = get_abstracts("aff(Repsol)", 300, "Elsevier_abstracts - Repsol")
Repsol

Artigos:  100
Artigos:  200


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/S0895-9811(01)00068-2,Pre-Silurian control in the genesis of the cen...,"[Giraudo, R., Limachi, R.]",2001-12-31,"[Pre-Silurian basement, Bolivia, Thrust-and-fo..."
1,,10.1016/j.rce.2011.09.012,Ser médico… «y ejercer a los 80 años»,"[Torres González, J.A.]",2012-01-31,[]
2,\n Abstract\n \n ...,10.1016/j.tecto.2014.02.009,Concave slab out board of the Tonga subduction...,"[Martin, A.K.]",2014-05-21,"[Opposite microplate rotations, Opposite toroi..."
3,\n Abstract\n \n ...,10.1016/S1089-3156(00)00026-X,Derivation of continuous molecular weight dist...,"[Whiteley, K.S., Garriga, A.]",2001-08-31,"[Molecular weight distribution, Laplace Transf..."
4,\n Abstract\n \n ...,10.1016/0098-1354(96)00276-1,Industrial application of a simulation model f...,"[Lopez, Antonio, Pedraza, Juan J., del Amo, Be...",1996-12-31,"[Low Density Polyethylene, Free-Radical Polyme..."
5,\n Abstract\n \n ...,10.1016/S0016-2361(96)00169-X,Characterization of petroleum bitumens and the...,"[Jiménez-Mateos, Juan Miguel, Quintero, Luis C...",1996-11-30,"[bitumen, thermal analysis, physicochemical pr..."
6,\n Publisher Summary\n \...,10.1016/B978-188420776-1.50002-8,Single-Site Supported Catalysts for Ethylene,"[Muñoz-Escalona, A., Mendez, L., Peña, B., Laf...",1999-12-31,[]
7,\n Mesozoic extension of the Grand ...,10.1016/B978-0-444-56356-9.00013-4,14 Extensional tectonics and stratigraphy of t...,"[Welsink, Herman, Tankard, Anthony]",2012-12-31,[]
8,\n Publisher Summary\n \...,10.1016/B978-044450611-5/50018-0,Chapter 16 Stratigraphy and hydrocarbon potent...,"[Aziz, A.]",2000-12-31,[]
9,\n Abstract\n \n ...,10.1016/0040-6031(95)02281-6,Determination of volatile material and ash con...,"[Rial, C., Jiménez-Mateos, J.M.]",1995-08-01,"[Ash content, Polymeric sulfur, TG, TGA, Volat..."


In [43]:
Statoil = get_abstracts("aff(Statoil)", 1000, "Elsevier_abstracts - Statoil")
Statoil

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.ifacol.2016.07.308,Robust Automatic Well Choke Control – Physical...,"[Kittilsen, Pål, Fjalestad, Kjetil, Sperle, In...",2016-12-31,"[automatic control, estimation, robust, constr..."
1,\n Abstract\n \n ...,10.1016/j.ijrmms.2005.04.005,Improved routine estimation of the minimum hor...,"[Raaen, A.M., Horsrud, P., Kjørholt, H., Øklan...",2006-01-31,"[In situ stress, Flowback, XLOT]"
2,\n Abstract\n \n ...,10.1016/B978-0-444-53813-0.00009-5,Chapter 9 Methodology and Techniques,"[Knaust, Dirk]",2012-12-31,"[Ichnological analysis, Ichnofabric, Bioturbat..."
3,\n Abstract\n \n ...,10.1016/j.energy.2004.03.074,CO2 underground storage for Snøhvit gas field ...,"[Maldal, T, Tappel, I.M]",2004-08-31,[]
4,\n Abstract\n \n ...,10.1016/B978-0-444-53813-0.00003-4,Chapter 3 Trace-Fossil Systematics,"[Knaust, Dirk]",2012-12-31,"[Trace-fossil classification, Ichnotaxonomy, N..."
5,\n Abstract\n \n ...,10.1016/S0920-4105(03)00023-8,Local rock mechanical knowledge improves drill...,"[Stjern, G, Agle, A, Horsrud, P]",2003-06-30,"[Wellbore stability, Shale, Drilling, Mud]"
6,\n \n Faulting ...,10.1016/S0920-5446(08)70255-3,Ostracod Zones and Dispersion of Mesozoic Foss...,"[Christensen, Ole Bruun]",1988-12-31,[]
7,\n \n Polymeriz...,10.1016/S0167-2991(08)61617-1,6. Kinetic Profile of Polymerization with Cr-O...,"[Follestad, Arild, Helleborg, Stein, Almquist,...",1990-12-31,[]
8,\n Publisher Summary\n ...,10.1016/S0167-2991(08)60111-1,Synthesis Gas Production,"[Solbakken, Åge]",1991-12-31,[]
9,\n Abstract\n \n ...,10.1016/0141-1187(85)90038-0,Wave climate off northern Norway,"[Haver, Sverre]",1985-04-30,[]


In [44]:
Equinor = get_abstracts("aff(Equinor)", 100, "Elsevier_abstracts - Equinor")
Equinor

,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.coldregions.2018.11.016,An iceberg forecast approach based on a statis...,"[Andersson, Leif Erik, Scibilia, Francesco, Im...",2019-02-28,"[Iceberg drift forecasting, Offshore operation..."
1,\n Abstract\n \n ...,10.1016/j.egypro.2018.07.021,The CCS hub in Norway: some insights from 22 y...,"[Ringrose, Philip S.]",2018-07-31,"[CO2 Storage, Saline aquifers, CCS, Storage Ca..."
2,\n Abstract\n \n ...,10.1016/j.ijggc.2018.11.008,Visualization of hydrate formation during CO2 ...,"[Almenningen, Stian, Gauteplass, Jarand, Fotla...",2018-12-31,"[Subsurface CO2 sequestration, Gas hydrate, MR..."
3,\n Abstract\n \n ...,10.1016/j.ress.2018.10.001,Lessons learned from applying a new HRA method...,"[Taylor, Claire, Øie, Sondre, Gould, Kristian]",2018-10-05,"[Human Reliability Analysis, HRA, Qualitative ..."
4,\n Abstract\n \n ...,10.1016/j.marpetgeo.2018.06.028,Tiran Straits: The transfer zone between the R...,"[Mart, Yossi, Hall, John K., Vachtman, Dina]",2018-11-30,"[Tiran Straits, Elat Rift, Northern red sea, O..."
5,\n Abstract\n \n ...,10.1016/j.ijrmms.2018.07.018,Three-dimensional bonded-particle discrete ele...,"[Park, Bona, Min, Ki-Bok, Thompson, Nicholas, ...",2018-10-31,"[Transversely isotropic rock, Discrete element..."
6,\n Abstract\n \n ...,10.1016/j.fuel.2018.06.077,Modelling of elemental mercury solubility in n...,"[Koulocheris, Vassilis, Louli, Vasiliki, Pante...",2018-12-01,"[Mercury, Solubility, UMR-PRU, Equation of state]"
7,\n Abstract\n \n ...,10.1016/j.marpetgeo.2018.08.022,Morphology and pattern of Quaternary sedimenta...,"[Ottesen, D., Batchelor, C.L., Dowdeswell, J.A...",2018-12-31,"[North Sea Basin, Seismic stratigraphy, Quater..."
8,\n Abstract\n \n ...,10.1016/j.marstruc.2018.08.001,Quasi-static and dynamic indentation of offsho...,"[Vestrum, Ole, Kristoffersen, Martin, Polanco-...",2018-11-30,[]
9,\n Abstract\n \n ...,10.1016/j.jsg.2018.10.010,"Fault deformation, seismic amplitude and unsup...","[Cunningham, Jennifer, Cardozo, Nestor, Townse...",2019-01-31,"[Faults, Dip distortion, Seismic attributes, S..."


In [45]:
British_Petroleum = get_abstracts("aff(British%20Petroleum)", 500, "Elsevier_abstracts - British Petroleum")
British_Petroleum

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/0264-8172(89)90058-5,Major problems in geology: an industrial persp...,"[Vann, I.R.]",1989-11-30,[]
1,,10.1016/0264-8172(92)90040-L,Source and migration processes and evaluation ...,"[Gibbons, M.J.]",1992-12-31,[]
2,Unknown,10.1016/B978-0-08-010385-3.50003-6,Foreword,"[SELLERS, E.S.]",1963-12-31,[]
3,,10.1016/0264-8172(91)90016-T,Petroleum geology of North Greenland \n ...,"[Hurst, John M.]",1991-05-31,[]
4,\n Publisher Summary\n \...,10.1016/B978-0-08-026801-9.50018-9,The Effects of Energy Development on East Euro...,"[Scanlan, Tony]",1980-12-31,[]
5,,10.1016/B978-1-4831-9829-3.50015-2,FUELS FOR AUTOMOTIVE DIESEL ENGINES: THE USER'...,"[STRETTEL, R.P., SAVAGE, J.D.]",1964-12-31,[]
6,,10.1016/0140-9883(79)90009-4,International studies of the demand for energy...,"[Drollas, L.P.]",1979-10-31,[]
7,,10.1016/0165-232X(83)90076-9,The impact of industrial development on the ar...,"[Larminie, Geoffrey]",1983-06-30,[]
8,\n Abstract\n \n ...,10.1016/0302-184X(78)90028-8,Ecological monitoring as a management tool in ...,"[Cowell, Eric B.]",1978-12-31,[]
9,,10.1016/0308-597X(77)90099-9,Petroleum resources of seas and oceans B.A. So...,"[Caston, V.N.D.]",1977-10-31,[]


In [46]:
BP_International = get_abstracts("aff(BP%20International)", 1600, "Elsevier_abstracts - BP International")
BP_International

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000
Artigos:  1100
Artigos:  1200
Artigos:  1300
Artigos:  1400
Artigos:  1500


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Publisher Summary\n \...,10.1016/B978-075067775-2/50021-4,Chapter 20 Smooth Operations,"[Peard, Nace S., Steen, Wiley, Stefanov, Mike]",2005-12-31,[]
1,\n ABSTRACT\n \n ...,10.1016/B978-0-85295-205-4.50012-8,EXPERT SYSTEM FOR PRODUCTION SEPARATOR DESIGN,"[Dick, D R, Zelouf, M]",1987-12-31,[]
2,,10.1016/B978-0-434-91370-1.50040-5,28 The petroleum industry and the protection o...,"[Westaway, M T]",1988-12-31,[]
3,,10.1016/S0923-2532(05)80140-1,Le dosage sérique de la cytokératine 19 par le...,"[Rastel, D]",1994-12-31,[]
4,\n Abstract\n \n ...,10.1016/0308-521X(95)93648-W,Nutrient depletion by food crops in Ghana and ...,"[Rhodes, E.R.]",1995-12-31,[]
5,\n Abstract\n \n ...,10.1016/0169-4758(88)90019-1,\n Loa loa — a neglected filaria,"[Pinder, M.]",1988-10-31,[]
6,\n \n Keywords. Artif...,10.1016/B978-0-08-036937-2.50017-5,REAL-TIME ARTIFICIAL INTELLIGENCE FOR PROCESS ...,"[Thomson, A.C.]",1989-12-31,[]
7,,10.1016/S2213-2600(13)70127-9,Response to “The unknown: respiratory effects ...,"[Heron, Richard JL]",2013-07-31,[]
8,\n KEYWORDS\n Welding pr...,10.1016/B978-0-08-030537-0.50022-3,User's Requirements and Control of Activities,"[Thomas, D.B.J.]",1983-12-31,[]
9,,10.1016/B978-0-407-01160-1.50017-8,13 PRACTICAL ASPECTS OF FEEDING PROTEIN TO DAI...,"[TWIGGE, J.R., VAN GILS, L.G.M.]",1984-12-31,[]


In [47]:
BP_America = get_abstracts("aff(BP%20America)", 200, "Elsevier_abstracts - BP America")
BP_America

Artigos:  100


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Publisher Summary\n \...,10.1016/B978-188420781-5.50012-7,Mixing of a Low Molecular Weight Additive in ...,"[Cheng, Jianya, Sanchez, Ronald, Bigio, David I.]",1999-12-31,[]
1,\n Abstract\n \n ...,10.1016/j.cattod.2007.05.001,The effects of gas-to-oil rate in ultra low su...,"[Hoekstra, George]",2007-09-30,"[Gas-to-oil rate, Ultra low sulfur diesel (uls..."
2,Unknown,10.1016/B978-0-12-800867-6.09990-8,Foreword,"[Stankovic, John A.]",2015-12-31,[]
3,,10.1016/j.gca.2006.06.481,Isotopic biogeochemical signals in sediments f...,"[Hayes, J.M., Villinski, J.C., Brassell, S.C.,...",2006-09-30,[]
4,\n Abstract\n \n ...,10.1016/j.apor.2016.03.001,Fatigue analysis of offshore well conductors: ...,"[Zakeri, Arash, Clukey, Edward C., Kebadze, El...",2016-04-30,"[Well conductor, Fatigue analysis, \n ..."
5,\n Abstract\n \n ...,10.1016/j.apor.2016.03.002,Fatigue analysis of offshore well conductors: ...,"[Zakeri, Arash, Clukey, Edward C., Kebadze, El...",2016-04-30,"[Well conductor, Fatigue analysis, \n ..."
6,\n Abstract\n \n ...,10.1016/j.apcata.2018.11.031,Rapid Transfer Hydrogenation of Acetophenone U...,"[Braden, Drew J., Cariou, Renan, Shabaker, Joh...",2018-11-30,"[transfer hydrogenation, ruthenium, homogeneou..."
7,\n Abstract\n \n ...,10.1016/j.orggeochem.2005.01.010,Molecular and stable isotope compositions of n...,"[Milkov, Alexei V.]",2005-05-31,[]
8,\n Abstract\n \n ...,10.1016/0749-6036(88)90204-2,\n In situ ellipsometric charact...,"[Collins, R.W.]",1988-12-31,[]
9,\n Abstract\n \n ...,10.1016/0032-3861(91)90229-C,Phase behaviour of blend systems containing ni...,"[Percec, Simona, Hammond, Terry]",1991-12-31,"[single-phase polymer blend, multicomponent ph..."


In [48]:
BP_Exploration = get_abstracts("aff(BP%20Exploration)", 400, "Elsevier_abstracts - BP Exploration")
BP_Exploration

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/S0264-8172(97)81120-8,Basin compartments and seals \n ...,"[Vear, A.]",1997-08-31,[]
1,,10.1016/S0264-8172(97)88317-1,Hydrocarbon habitat in rift basins Geological ...,"[Roberts, David G.]",1997-02-28,[]
2,,10.1016/S0264-8172(97)81118-X,Modeling the earth for oil exploration \n ...,"[Raikes, Sue]",1997-08-31,[]
3,,10.1016/0264-8172(88)90012-8,Guide to the petroleum reference literature \n...,"[Wilson, John M.]",1988-08-31,[]
4,,10.1016/0264-8172(94)90062-0,Arctic Geology and Petroleum Potential: Procee...,"[Roberts, D.G.]",1994-06-30,[]
5,Unknown,10.1016/0264-8172(94)90018-3,Giant oil and gas fields of the decade 1978–88...,"[Roberts, D.G.]",1994-02-28,[]
6,,10.1016/0264-8172(94)90086-8,Gulf of Mexico basin \n A. Salva...,"[Roberts, D.G.]",1994-08-31,[]
7,,10.1016/0264-8172(94)90087-6,Sedimentology and sequence stratigraphy of ree...,"[Emery, D.]",1994-08-31,[]
8,,10.1016/0264-8172(92)90041-C,Undiscovered oil and gas resources: An evaluat...,"[Harper, F.]",1992-12-31,[]
9,,10.1016/0264-8172(96)83691-9,Computing risk for oil prospects: Principles a...,"[Harper, Francis]",1996-11-30,[]


In [49]:
BP_United_Kingdom = get_abstracts("aff(BP%20United%20Kingdom)", 800, "Elsevier_abstracts - BP United Kingdom")
BP_United_Kingdom

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.fuel.2018.06.084,Surface pre-ignition measurements of fuel comp...,"[Mutzke, Johannes, Stone, Richard, Williams, J...",2018-12-01,"[Pre-ignition, Abnormal combustion, Fuel compo..."
1,\n \n Keywords Seismi...,10.1016/B978-0-444-53042-4.00010-8,10 2D and 3D seismic data,"[Redshaw, T.C.]",2012-12-31,[]
2,\n Abstract\n \n ...,10.1016/j.msea.2016.12.088,Effect of hydrogen on the mechanical propertie...,"[Demetriou, V., Robson, J.D., Preuss, M., Mora...",2017-01-27,"[Alloy 945X, Hydrogen Embrittlement, δ phase, ..."
3,\n Abstract\n \n ...,10.1016/j.jvolgeores.2010.02.009,A model of overturn of CO2 laden lakes trigger...,"[Mott, R.W, Woods, A.W]",2010-05-10,"[CO2\n , Nyos, Monoun, bubble..."
4,\n Abstract\n \n ...,10.1016/j.epsl.2009.09.015,Magma mixing triggered during volcanic eruptions,"[Woods, Andrew W., Cowan, Alexander]",2009-10-30,"[magma, mixing, volcanic eruptions]"
5,\n Abstract\n \n ...,10.1016/j.engfracmech.2017.12.010,Estimation of crack initiation stress and loca...,"[Sampath, Dhinakaran, Akid, Robert, Morana, Ro...",2018-03-15,"[Ni-alloys 945X and 718, SSRT, Hydrogen chargi..."
6,,10.1016/j.nbt.2014.05.1620,Designing robust Saccharomyces cerevisiae stra...,"[Kumar, Vinod, Greetham, Darren, Wimalasena, T...",2014-07-31,[]
7,\n Abstract\n \n ...,10.1016/S0010-2180(97)00329-5,Correlations for the Laminar-Burning Velocity ...,"[Stone, R., Clarke, A., Beckwith, P.]",1998-09-30,[]
8,\n Abstract\n \n ...,10.1016/0010-2180(87)90054-X,Combustion in crater beds,"[Malik, T.I., Weinberg, F.J., Boden, J.C., Ful...",1987-05-31,[]
9,\n Abstract\n \n ...,10.1016/j.apcata.2017.08.017,\n In situ X-ray diffraction of ...,"[Paterson, James, Peacock, Mark, Ferguson, Ewe...",2017-09-25,"[Cobalt oxide, Reduction, Water, \n ..."


Extrair artigos por ano

In [50]:
abstract_2017 = get_abstracts("date(2017)&sortBy=relevance", 1000, "Elsevier_abstracts - 2017")
abstract_2017

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/B978-0-323-24288-2.00049-0,Chapter 49 Dream Content Quantitative Findings,"[Zadra, Antonio, Domhoff, G. William]",2017-12-31,[]
1,,10.1016/B978-0-323-24288-2.00006-4,Chapter 6 Genetics of Normal Human Sleep,"[Hor, Hyun, Tafti, Mehdi]",2017-12-31,[]
2,,10.1016/B978-0-323-24288-2.00018-0,Chapter 18 Respiratory Physiology Sleep at Hig...,"[Ainslie, Philip N., Burgess, Keith R.]",2017-12-31,[]
3,,10.1016/B978-0-323-39038-5.00024-X,Chapter 24 Principles of Fracture Fixation Pla...,"[von Keudell, Arvind, Collins, Michael, Jupite...",2017-12-31,[]
4,,10.1016/B978-0-323-24288-2.00065-9,Chapter 65 Sleep Forensics Criminal Culpabilit...,"[Cramer-Bornemann, Michel A., Mahowald, Mark W.]",2017-12-31,[]
5,,10.1016/B978-0-323-24288-2.00092-1,Chapter 92 Parkinsonism,"[Trenkwalder, Claudia, Arnulf, Isabelle, Postu...",2017-12-31,[]
6,,10.1016/B978-0-323-24288-2.00112-4,Chapter 112 Snoring and Pathologic Upper Airwa...,"[Stoohs, Riccardo, Gold, Avram R.]",2017-12-31,[]
7,,10.1016/B978-0-323-24288-2.00080-5,Chapter 80 Insomnia Recent Developments and Fu...,"[Buysse, Daniel J., Harvey, Allison G.]",2017-12-31,[]
8,,10.1016/B978-0-323-24288-2.00147-1,Chapter 147 Oral Appliances for the Treatment ...,"[Lettieri, Christopher J., Almeida, Fernanda R...",2017-12-31,[]
9,,10.1016/B978-0-323-24288-2.00058-1,Chapter 58 Cardinal Manifestations of Sleep Di...,"[Vaughn, Bradley V., D'Cruz, O'Neill F.]",2017-12-31,[]


In [51]:
abstract_2016 = get_abstracts("date(2016)&sortBy=relevance", 1000, "Elsevier_abstracts - 2016")
abstract_2016

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/B978-0-7020-5554-6.00009-5,Chapter 9 Pathology,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
1,Unknown,10.1016/B978-0-7020-5554-6.00010-1,Front Matter,[],2016-12-31,[]
2,,10.1016/B978-0-7020-5554-6.00001-0,Chapter 1 Anatomy of the eye and orbit,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
3,,10.1016/B978-0-7020-5554-6.00002-2,Chapter 2 Embryology and early development of ...,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
4,Unknown,10.1016/B978-0-7020-5554-6.00011-3,Copyright,[],2016-12-31,[]
5,Unknown,10.1016/B978-0-7020-5554-6.00012-5,Preface,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
6,,10.1016/B978-0-7020-5554-6.00005-8,Chapter 5 Physiology of vision and the visual ...,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
7,,10.1016/B978-0-7020-5554-6.00006-X,Chapter 6 General and ocular pharmacology,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
8,,10.1016/B978-0-7020-5554-6.00007-1,Chapter 7 Immunology,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]
9,,10.1016/B978-0-7020-5554-6.00004-6,Chapter 4 Biochemistry and cell biology,"[Forrester, John V., Dick, Andrew D., McMenami...",2016-12-31,[]


In [52]:
abstract_2015 = get_abstracts("date(2015)&sortBy=relevance", 1000, "Elsevier_abstracts - 2015")
abstract_2015

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.proeng.2015.06.191,Development of Grout for Additional Seal Emban...,"[Kociánová, Magdaléna, Černý, Vít, Drochytka, ...",2015-12-31,"[additional sealing earth dams, injection, gro..."
1,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.007,The Elasto-plastic Response of the Bone Tissue...,"[Tavares, C.S.S., Belinha, J., Dinis, L.M.J.S....",2015-12-31,"[Radial Point Interpolation Method (RPIM), Nat..."
2,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.015,Design and Validation of an Open-Hardware Prin...,"[De Maria, Carmelo, Ferrari, Laura, Montemurro...",2015-12-31,"[Inkjet, Piezoelectric, Bioprinting, Opensource.]"
3,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.016,Neuro-muscular Regeneration Using Scaffolds wi...,"[Caseiro, Ana Rita, Pereira, Tiago, Ribeiro, J...",2015-12-31,"[nerve regeneration, neurogenic muscle atrophy..."
4,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.017,The Study of Differentiation of Oral Fibroblas...,"[Jang, Hyun-Seon, Lee, Mi-Sa]",2015-12-31,"[Fibroblast, Differentiation, S-100]"
5,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.018,The Study of Media Effect on Differentiation o...,"[Lee, IL-Kwon, Jang, Hyun-Seon]",2015-12-31,"[Fibroblast, Differentiation, Media]"
6,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.019,Combination of 3D Extruded-based Poly (ɛ-capro...,"[Moura, Carla Sofia, Silva, Cláudia Lobato da,...",2015-12-31,"[Mesenchymal Stem/Stromal Cells, Poly (ɛ-capro..."
7,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.020,Hybrid Fabrication of a 3D Printed Geometry Em...,"[Mendoza-Buenrostro, Christian, Lara, Hernan, ...",2015-12-31,"[Scaffolds, nanofibers, 3D printing, fused dep..."
8,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.056,Computer Modelling of Precast Large-panel Buil...,"[Barański, Jacek, Berkowski, Piotr]",2015-12-31,"[Large-panel precast buildings, FEM modelling,..."
9,\n Abstract\n \n ...,10.1016/j.proeng.2015.07.057,The Analysis of Slab and Beam Floor with Pre-s...,"[Panfilov, Denis A., Pischulev, Alexander A.]",2015-12-31,"[Reinforced concrete structures, Slab and beam..."


In [53]:
abstract_2014 = get_abstracts("date(2014)&sortBy=relevance", 1000, "Elsevier_abstracts - 2014")
abstract_2014

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n \n Abstract\n ...,10.1016/B978-0-12-407179-7.00002-3,Chapter 2 Discovery of the T- and B-Cell Compa...,"[Cooper, Max D.]",2014-12-31,"[working with Bob Good, bursa of Fabricius, bu..."
1,\n \n Abstract\n ...,10.1016/B978-0-12-407179-7.00004-7,Chapter 4 From Immunodeficiency to Autoimmunity,"[Notarangelo, Luigi Daniele]",2014-12-31,"[immunodeficiency, autoimmunity, Omenn syndrom..."
2,\n \n Abstract\n ...,10.1016/B978-0-12-407179-7.00005-9,Chapter 5 Immunological Tests – from the Micro...,"[Fleisher, Thomas A.]",2014-12-31,"[diagnostic immunology, laboratory testing, hu..."
3,Unknown,10.1016/B978-0-12-405948-1.00024-4,Copyright,[],2014-12-31,[]
4,\n \n Abstract\n ...,10.1016/B978-0-12-407179-7.00003-5,Chapter 3 Evolution of the Definition of Prima...,"[Picard, Capucine, Casanova, Jean-Laurent]",2014-12-31,"[primary immunodeficiency, agammaglobulinemia,..."
5,\n \n Abstract\n ...,10.1016/B978-0-12-407179-7.00009-6,Chapter 9 Wiskott–Aldrich Syndrome: from a Fat...,"[Ochs, Hans D., Belohradsky, Bernd H.]",2014-12-31,"[discovery of WAS, XLT and X-linked neutropeni..."
6,\n Abstract\n \n ...,10.1016/B978-0-12-407171-1.00005-8,Chapter 5 Mobile Search Ranking,"[Long, Bo, Chang, Yi]",2014-12-31,"[Local search, mobile local search, ranking si..."
7,\n Abstract\n \n ...,10.1016/B978-0-12-407171-1.00006-X,Chapter 6 Entity Ranking,"[Long, Bo, Chang, Yi]",2014-12-31,"[Entity ranking, structured data, object ranking]"
8,\n Abstract\n \n ...,10.1016/B978-0-12-407171-1.00007-1,Chapter 7 Multi-Aspect Relevance Ranking,"[Long, Bo, Chang, Yi]",2014-12-31,"[Vertical search, multi-aspect relevance, mapp..."
9,\n Abstract\n \n ...,10.1016/B978-0-12-407171-1.00008-3,Chapter 8 Aggregated Vertical Search,"[Long, Bo, Chang, Yi]",2014-12-31,"[Aggregated search, vertical selection, vertic..."


In [54]:
abstract_2013 = get_abstracts("date(2013)&sortBy=relevance", 1000, "Elsevier_abstracts - 2013")
abstract_2013

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n The retard...,10.1016/B978-0-12-407177-3.00005-2,Chapter 5 Real Time Green’s Functions and Temp...,"[Kitamura, Toyoyuki]",2013-12-31,[]
1,\n Abstract\n The quantu...,10.1016/B978-0-12-407177-3.00003-9,Chapter 3 Fundamentals of Quantum Field Theory,"[Kitamura, Toyoyuki]",2013-12-31,[]
2,,10.1016/B978-0-12-407173-5.09989-1,Contributors,[],2013-12-31,[]
3,,10.1016/B978-0-12-407173-5.09990-8,Index,[],2013-12-31,[]
4,,10.1016/B978-0-12-407173-5.09993-3,Copyright,[],2013-12-31,[]
5,\n Abstract\n A brief su...,10.1016/B978-0-12-407177-3.00001-5,Chapter 1 Introduction,"[Kitamura, Toyoyuki]",2013-12-31,[]
6,\n Abstract\n The discri...,10.1016/B978-0-12-407177-3.00002-7,Chapter 2 Sound and Elastic Waves in the Class...,"[Kitamura, Toyoyuki]",2013-12-31,[]
7,"\n In this chapter, we present the ...",10.1016/B978-0-12-407164-3.00008-5,"9 Coherent, Elastic Scatter of Neutrons by Ato...","[Cremer, Jay Theodore]",2013-12-31,[]
8,"\n First, this chapter derives the ...",10.1016/B978-0-12-407164-3.00010-3,"11 Kirchhoff Equation Solution for CRL, Pinhol...","[Cremer, Jay Theodore]",2013-12-31,[]
9,Unknown,10.1016/B978-0-12-407157-5.00005-1,Copyright,[],2013-12-31,[]


In [55]:
abstract_2012 = get_abstracts("date(2012)&sortBy=relevance", 1000, "Elsevier_abstracts - 2012")
abstract_2012

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/B978-0-12-407178-0.00005-3,Chapter Four The Role of Serotonin in Axon and...,"[Trakhtenberg, Ephraim F., Goldberg, Jeffrey L.]",2012-12-31,"[Axon regeneration, Neurite growth, Axon growt..."
1,\n Abstract\n \n ...,10.1016/B978-0-12-407178-0.00006-5,Chapter Five Inflammatory Pathways in Spinal C...,"[David, Samuel, Zarruk, Juan Guillermo, Ghasem...",2012-12-31,"[Spinal cord injury, Inflammation, Secondary d..."
2,\n Abstract\n \n ...,10.1016/B978-0-12-407178-0.00007-7,Chapter Six Combinatorial Therapy Stimulates L...,"[de Lima, Silmara, Habboub, Ghaith, Benowitz, ...",2012-12-31,"[Optic nerve injury, Long-term regeneration, C..."
3,\n Abstract\n \n ...,10.1016/B978-0-12-407178-0.00008-9,Chapter Seven From Bench to Beside to Cure Spi...,"[Hug, Andreas, Weidner, Norbert]",2012-12-31,"[Spinal cord injury, Regeneration clinical tri..."
4,,10.1016/B978-0-12-407178-0.09986-5,Preface,"[Goldberg, Jeffrey L., Trakhtenberg, Ephraim F.]",2012-12-31,[]
5,,10.1016/B978-0-12-407178-0.09990-7,Contributors,[],2012-12-31,[]
6,,10.1016/B978-0-12-407178-0.09992-0,Copyright,[],2012-12-31,[]
7,,10.1016/B978-0-12-407178-0.09993-2,Contents of Recent Volumes,[],2012-12-31,[]
8,\n Abstract\n \n ...,10.1016/B978-0-12-407178-0.00002-8,Chapter One Neurotrophic Factors and the Regen...,"[Harvey, Alan R., Ooi, Jacob Wei Wei, Rodger, ...",2012-12-31,"[Retinal ganglion cells, Optic nerve, Optic tr..."
9,,10.1016/B978-0-12-407178-0.09988-9,Series Page,[],2012-12-31,[]


In [56]:
abstract_2011 = get_abstracts("date(2011)&sortBy=relevance", 1000, "Elsevier_abstracts - 2011")
abstract_2011

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/S0169-7218(11)00411-4,Chapter 5 Program Evaluation and Research Desi...,"[DiNardo, John, Lee, David S.]",2011-12-31,[]
1,,10.1016/S0169-7218(11)00401-1,Copyright,[],2011-12-31,[]
2,,10.1016/S0169-7218(11)00402-3,Contents of Volume 4A,[],2011-12-31,[]
3,\n Abstract\n \n ...,10.1016/S0169-7218(11)02412-9,Chapter 14 Local Labor Markets*\n ...,"[Enrico, Moretti]",2011-12-31,"[Cities, Wage, General equilibrium, Spatial eq..."
4,,10.1016/S0169-7218(11)02418-X,Chapter 20 Personnel Economics: Hiring and Inc...,"[Paul, Oyer, Scott, Schaefer]",2011-12-31,[]
5,,10.1016/S0169-7218(11)02419-1,Subject index to volume 4b,[],2011-12-31,[]
6,,10.1016/S0169-7218(11)02420-8,Subject index to volume 4a,[],2011-12-31,[]
7,\n Abstract\n \n ...,10.1016/S0169-7218(11)04114-1,Chapter 8 Extrinsic Rewards and Intrinsic Moti...,"[Rebitzer, James B., Taylor, Lowell J.]",2011-12-31,"[Principal agent models, Intrinsic and extrins..."
8,\n Publisher Summary\n ...,10.1016/S0169-7218(11)00414-X,Subject Index to Volume 4A,[],2011-12-31,[]
9,,10.1016/S0169-7218(11)02402-6,Front matter,[],2011-12-31,[]


In [ ]:
abstract_2010 = get_abstracts("date(2010)&sortBy=relevance", 1000, "Elsevier_abstracts - 2010")
abstract_2010

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/S0169-7218(10)02010-1,Chapter 26 Using Innovation Surveys for Econom...,"[Mairesse, Jacques, Mohnen, Pierre]",2010-12-31,"[collaboration, complementarity, econometrics,..."
1,,10.1016/S0169-7218(10)02016-2,Series Page,[],2010-12-31,[]
2,\n Publisher Summary\n \...,10.1016/B978-8-1312-2297-3.50005-9,Chapter 5 Protein Structure Prediction,"[Gromiha, M. Michael]",2010-12-31,[]
3,Unknown,10.1016/B978-8-1312-2297-3.50012-6,Preface,"[Gromiha, M. Michael]",2010-12-31,[]
4,\n Publisher Summary\n \...,10.1016/B978-8-1312-2297-3.50001-1,Chapter 1 Proteins,"[Gromiha, M. Michael]",2010-12-31,[]
5,\n Publisher Summary\n \...,10.1016/B978-8-1312-2297-3.50006-0,Chapter 6 Protein Stability,"[Gromiha, M. Michael]",2010-12-31,[]
6,\n Publisher Summary\n \...,10.1016/B978-8-1312-2297-3.50007-2,Chapter 7 Protein Interactions,"[Gromiha, M. Michael]",2010-12-31,[]
7,Unknown,10.1016/B978-8-1312-2297-3.50008-4,Front matter,[],2010-12-31,[]
8,Unknown,10.1016/B978-8-1312-2297-3.50009-6,Copyright,[],2010-12-31,[]
9,Unknown,10.1016/B978-8-1312-2297-3.50010-2,Dedication,[],2010-12-31,[]


In [6]:
abstract_2009 = get_abstracts("date(2009)&sortBy=relevance", 1000, "Elsevier_abstracts - 2009")
abstract_2009

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/S0025-326X(09)00420-2,SI: Title and Editorial Board,[],2009-12-31,[]
1,,10.1016/S0025-326X(09)00262-8,SI: Title and Editorial Board,[],2009-12-31,[]
2,,10.1016/B978-84-458-1938-8.50014-3,Capítulo 14 Coberturas alternativas de las her...,"[Sheridan, Robert L., Tompkins, Ronald G.]",2009-12-31,[]
3,,10.1016/B978-84-458-1938-8.50015-5,Capítulo 15 AlloDerm,"[Barret, Juan P.]",2009-12-31,[]
4,,10.1016/B978-84-458-1938-8.50016-7,Capítulo 16 Fisiopatología de la lesión por in...,"[Traber, Daniel L., Herndon, David N., Enkhbaa...",2009-12-31,[]
5,,10.1016/B978-84-458-1938-8.50017-9,Capítulo 17 Diagnóstico y tratamiento de la le...,"[Nugent, Nora, Herndon, David N.]",2009-12-31,[]
6,,10.1016/B978-84-458-1938-8.50018-0,Capítulo 18 Tratamiento respiratorio,"[Mlcak, Ronald P., Herndon, David N.]",2009-12-31,[]
7,,10.1016/B978-84-458-1938-8.50019-2,Capítulo 19 Efectos de las quemaduras sobre el...,"[Klein, Gordon L., Przkora, Rene, Herndon, Dav...",2009-12-31,[]
8,,10.1016/B978-84-458-1938-8.50020-9,Capítulo 20 Homeostasis de las vitaminas y los...,"[Klein, Gordon L., Przkora, Rene, Herndon, Dav...",2009-12-31,[]
9,,10.1016/B978-84-458-1938-8.50039-8,Capítulo 39 Reconstrucción de la cabeza y el c...,"[Donelan, Matthias B.]",2009-12-31,[]


In [7]:
abstract_2008 = get_abstracts("date(2008)&sortBy=relevance", 1000, "Elsevier_abstracts - 2008")
abstract_2008

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Estos 43 casos clínicos ilustran...,10.1016/B978-84-458-1870-1.50026-0,Casos clínicos,"[Hamosh, Ada, McInnes, Roderick R., Nussbaum, ...",2008-12-31,[]
1,\n Es necesario un conocimiento sól...,10.1016/B978-84-458-1898-5.50001-8,Capítulo 1 Estructura y función de la piel,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
2,"\n Con frecuencia, debido al lengua...",10.1016/B978-84-458-1898-5.50002-X,Capítulo 2 Evaluación de niños con enfermedade...,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
3,\n Los artrópodos están constanteme...,10.1016/B978-84-458-1898-5.50007-9,Capítulo 7 Infestaciones,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
4,"\n Las urticarias, los eritemas y l...",10.1016/B978-84-458-1898-5.50014-6,"Capítulo 14 Reacciones vasculares, urticaria, ...","[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
5,\n La pérdida de pelo es bastante m...,10.1016/B978-84-458-1898-5.50015-8,Capítulo 15 Trastornos del pelo,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
6,\n Los trastornos ungueales no son ...,10.1016/B978-84-458-1898-5.50016-X,Capítulo 16 Trastornos ungueales,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
7,\n El color de la piel de un lactan...,10.1016/B978-84-458-1898-5.50017-1,Capítulo 17 Trastornos de la pigmentación Lesi...,"[Weston, William L., Lane, Alfred T., Morelli,...",2008-12-31,[]
8,\n Abstract\n \n ...,10.1016/j.marpolbul.2008.01.039,Interelement relationships and age-related var...,"[Agusa, Tetsuro, Nomura, Kumiko, Kunito, Takas...",2008-12-31,"[Trace elements, Mercury, Selenium, Age, Strip..."
9,\n Abstract\n \n ...,10.1016/j.marpolbul.2008.01.033,"Field validation, in Scotland and Iceland, of ...","[Leung, Kenneth M.Y., Furness, Robert W., Svav...",2008-12-31,"[Environmental monitoring, Mussel watch, Envir..."


In [8]:
abstract_2007 = get_abstracts("date(2007)&sortBy=relevance", 1000, "Elsevier_abstracts - 2007")
abstract_2007

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Abstract\n \n ...,10.1016/j.marpolbul.2007.09.014,Toward the application of ecological concepts ...,"[de Jonge, Victor N.]",2007-12-31,[]
1,\n Abstract\n \n ...,10.1016/j.marpolbul.2007.09.004,Sulphate reduction in the sediment of the Veni...,"[Zaggia, Luca, Rosso, Jessica, Zonta, Roberto]",2007-12-31,"[Anoxic sediment, Sulphate reduction, Sulphur ..."
2,\n Abstract\n \n ...,10.1016/j.marpolbul.2007.01.010,Assessment of biopollution in aquatic ecosystems,"[Olenin, Sergej, Minchin, Dan, Daunys, Darius]",2007-12-31,"[Xenodiversity, Biological pollution, Communit..."
3,\n \n \n ...,10.1016/B978-84-458-1717-9.50024-X,Capítulo 13 Neuropatías por atrapamiento,"[Chammas, M., Coulet, B.]",2007-12-31,[]
4,\n \n Guía de lectura...,10.1016/B978-84-458-1770-4.50016-8,Capítulo 16 Principios de la cirugía otoneurol...,"[Bordure, Philippe, Robier, Alain, Malard, Oli...",2007-12-31,[]
5,\n \n Guía de lectura...,10.1016/B978-84-458-1770-4.50017-X,Capítulo 17 Vía translaberíntica. Exéresis del...,"[Bordure, Philippe, Robier, Alain, Malard, Oli...",2007-12-31,[]
6,\n \n Guía de lectura...,10.1016/B978-84-458-1770-4.50018-1,Capítulo 18 Vía suprapetrosa. Vía de la fosa c...,"[Bordure, Philippe, Robier, Alain, Malard, Oli...",2007-12-31,[]
7,\n \n Guía de lectura...,10.1016/B978-84-458-1770-4.50019-3,Capítulo 19 Vía retrolaberíntica,"[Bordure, Philippe, Robier, Alain, Malard, Oli...",2007-12-31,[]
8,\n \n Esta intervenci...,10.1016/B978-84-458-1773-5.50013-X,Capítulo 13 Exéresis de los segmentos 4 y 5 o ...,"[Castaing, Denis, Azoulay, Daniel, Adam, René]",2007-12-31,[]
9,,10.1016/B978-84-458-1773-5.50014-1,Capítulo 14 Exéresis de varios segmentos u otr...,"[Castaing, Denis, Azoulay, Daniel, Adam, René]",2007-12-31,[]


In [9]:
abstract_2006 = get_abstracts("date(2006)&sortBy=relevance", 1000, "Elsevier_abstracts - 2006")
abstract_2006

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50009-9,Chapter 8 Convex sets and convex cones,"[Sinha, S.M.]",2006-12-31,[]
1,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50008-7,Chapter 7 Systems of linear equations and line...,"[Sinha, S.M.]",2006-12-31,[]
2,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50022-1,Chapter 21 The decomposition principle for lin...,"[Sinha, S.M.]",2006-12-31,[]
3,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50023-3,Chapter 22 Polynomial time algorithms for line...,"[Sinha, S.M.]",2006-12-31,[]
4,\n We recall that the general probl...,10.1016/B978-813120376-7/50024-5,Chapter 23 Nonlinear programming,"[Sinha, S.M.]",2006-12-31,[]
5,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50025-7,Chapter 24 Quadratic programming,"[Sinha, S.M.]",2006-12-31,[]
6,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50017-8,Chapter 16 Variants of the simplex method,"[Sinha, S.M.]",2006-12-31,[]
7,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50018-X,Chapter 17 Post-optimization problems Sensitiv...,"[Sinha, S.M.]",2006-12-31,[]
8,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50019-1,Chapter 18 Bounded variable problems,"[Sinha, S.M.]",2006-12-31,[]
9,\n Publisher Summary\n \...,10.1016/B978-813120376-7/50020-8,Chapter 19 Transportation problems,"[Sinha, S.M.]",2006-12-31,[]


In [10]:
abstract_2005 = get_abstracts("date(2005)&sortBy=relevance", 1000, "Elsevier_abstracts - 2005")
abstract_2005

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n \n Abstract\...,10.1016/S0169-7161(04)24002-0,2 From Data Mining to Knowledge Mining,"[Kaufman, Kenneth A., Michalski, Ryszard S.]",2005-12-31,[]
1,\n \n The basic...,10.1016/S0169-7161(05)25004-6,Model Selection and Hypothesis Testing based o...,"[Pericchi, Luis Raúl]",2005-12-31,"[Bayes factors, Bayesian information criterion..."
2,\n \n We review...,10.1016/S0169-7161(05)25005-8,Role of P-values and other Measures of Evidenc...,"[Ghosh, Jayanta, Purkayastha, Sumitra, Samanta...",2005-12-31,"[Bayes factors, Bayesian P-values, Pitman alte..."
3,\n Publisher Summary\n ...,10.1016/S0169-7161(04)24017-2,17 Data Visualization and Virtual Reality,"[Chen, Jim X.]",2005-12-31,[]
4,,10.1016/S0169-7161(04)24018-4,Preface,"[Rao, C.R., Wegman, E.J., Solka, J.L.]",2005-12-31,[]
5,\n \n In the se...,10.1016/S0169-7161(05)25020-4,Bayesian Thinking in Spatial Statistics,"[Waller, Lance A.]",2005-12-31,[]
6,,10.1016/S0169-7161(04)24020-2,Contributors,[],2005-12-31,[]
7,,10.1016/S0169-7161(04)24021-4,Subject Index,[],2005-12-31,[]
8,,10.1016/S0169-7161(04)24511-4,Series Page,[],2005-12-31,[]
9,,10.1016/S0169-7161(04)24512-6,Copyright,[],2005-12-31,[]


In [11]:
abstract_2004 = get_abstracts("date(2004)&sortBy=relevance", 1000, "Elsevier_abstracts - 2004")
abstract_2004

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Summary\n \n ...,10.1078/1434-8411-54100249,A Centralized MAC-Level Admission Control Algo...,"[Choi, Woo-Yong]",2004-12-31,"[IEEE 802.11, MAC, Traffic Stream, Admission c..."
1,\n Summary\n \n ...,10.1078/1434-8411-54100264,A Computer-Aided Design Technique for Lossless...,"[Sertbas, Ahmet, Yarman, B. Siddik]",2004-12-31,"[Lumped-Distributed, Matching networks, CAD]"
2,\n Summary\n \n ...,10.1078/1434-8411-54100266,A Novel High CMRR High Input Impedance Differe...,"[Ibrahim, Muhammed A., Kuntman, Hakan]",2004-12-31,"[CMRR, KHN-biquad, DDCC]"
3,\n Summary\n \n ...,10.1078/1434-8411-54100267,Air Cavity Incorporation to LTCC Spiral Induct...,"[Eun, Ki Chan, Park, Chul Soon]",2004-12-31,"[Low Temperature co-fired ceramic (LTCC), Spir..."
4,\n Summary\n \n ...,10.1078/1434-8411-54100226,Quality of Service in VoIP Communication,"[Uhl, Tadeus]",2004-12-31,"[Real time communication, Multimedia applicati..."
5,\n Summary\n \n ...,10.1078/1434-8411-54100227,Aspects of Switching System Analysis a Centuri...,"[Burda, Ralf, Folgmann, Manfred, Michaelis, St...",2004-12-31,"[Traffic theory, Network analysis, Data visual..."
6,\n Summary\n \n ...,10.1078/1434-8411-54100228,Performance Analysis of the ATM Adaptation Lay...,"[Ko, Gwangzeen, Moon, Sungdon, Ahmad, Aftab, K...",2004-12-31,"[ATM adaptation layer, AAL2, Performance Evalu..."
7,\n Summary\n \n ...,10.1078/1434-8411-54100229,On Convolutional Coupled Codes,"[Chaoui, Slim]",2004-12-31,"[Generator matrix, Minimum distance, Effective..."
8,\n Summary\n \n ...,10.1078/1434-8411-54100247,Robust Space Time Receiver for CDMA-based Ante...,"[Cho, Munhyoung, Lee, Chong Hyun, Kim, Soohong...",2004-12-31,"[Space-time receiver, CDMA, Projection, Signal..."
9,\n Summary\n \n ...,10.1078/1434-8411-54100257,An Adaptive Approach to Singular Point Detecti...,"[Rahimi, Mohammad Reza, Pakbaznia, Ehsan, Kasa...",2004-12-31,"[Fingerprint classification, Adaptive singular..."


In [12]:
abstract_2003 = get_abstracts("date(2003)&sortBy=relevance", 1000, "Elsevier_abstracts - 2003")
abstract_2003

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Summary\n \n ...,10.1078/1434-8411-54100183,Deterministic Maximum Likelihood Approach for ...,"[Kabaoğlu, Nihat, Çırpan, Hakan A., Çekli, Erd...",2003-12-31,"[Maximum Likelihood estimation, Antenna arrays..."
1,\n Abstract\n \n ...,10.1078/1434-8411-54100176,Multifunction CM/VM Biquads Realized with a Si...,"[Sharma, R.K., Senani, R.]",2003-12-31,"[Analog electronics, Circuit theory and design..."
2,\n \n Abstract\...,10.1078/1434-8411-54100177,Sequence Estimation with Transmit Diversity fo...,"[Panayırcı, Erdal, Aygölü, Ümit, Pusane, Ali E...",2003-12-31,"[Sequence estimation, Transmit diversity, EM a..."
3,\n \n Abstract\...,10.1078/1434-8411-54100178,Joint Channel Tracking and Symbol Detection fo...,"[Şen, Adnan, Çırpan, Hakan A., Panayırcı, Erdal]",2003-12-31,"[OFDM Systems, Joint channel estimation and sy..."
4,\n Summary\n \n ...,10.1078/1434-8411-54100172,Frequency and Timing Offset Estimation Using S...,"[Nikolic, Zorica, Stankovic, Veljko, Peric, Zo...",2003-12-31,"[OFDM, Synchronization, Frequency offset estim..."
5,\n \n Abstract\...,10.1078/1434-8411-54100173,Novel Grounded Parallel Immittance Simulator T...,"[Çam, Uğur, Kaçar, Fırat, Cicekoglu, Oguzhan, ...",2003-12-31,"[Immittance simulators, OTRA, Analog filters]"
6,\n Abstract\n \n ...,10.1078/1434-8411-54100192,On EBG Structures for Cellular Phone Applicati...,"[Bilotti, Filiberto, Vegni, Lucio, Viviani, Fr...",2003-12-31,"[Patch antennas, EBG, High impedance ground-pl..."
7,\n \n Abstract\...,10.1078/1434-8411-54100193,Analysis of Nonpreemptive Priority Queueing of...,"[Soo, Hooi Miin, Chung, Jong-Moon]",2003-12-31,"[Differentiated services (DiffServ), M/M/m/K, ..."
8,\n \n Abstract\...,10.1078/1434-8411-54100194,The Analysis of the Biconical Antenna by the L...,"[Jovićević, Svetozar, Jovanović, Ana]",2003-12-31,"[Biconical antennas, the Least-squares boundar..."
9,\n Abstract\n \n ...,10.1078/1434-8411-54100195,A Universal Shifter with Packed Data Formats,"[Jeong, Woo-Kyeong, Lee, Yong-Surk]",2003-12-31,"[Universal shifter, Barrel shifter, SIMD, Pack..."


In [13]:
abstract_2002 = get_abstracts("date(2002)&sortBy=relevance", 1000, "Elsevier_abstracts - 2002")
abstract_2002

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Summary\n \n ...,10.1078/1434-8411-54100110,Estimates of the Wigner Distribution in Gaussi...,"[Djurović, Igor, Stanković, LJubiša, Böhme, Jo...",2002-12-31,"[Time-frequency analysis, Wigner distribution,..."
1,\n Summary\n \n ...,10.1078/1434-8411-54100112,Algebraic Design of Some Equiripple Linear-Pha...,"[Lutovac, Budimir M., Lutovac, Miroslav D.]",2002-12-31,"[Optimal FIR filter, Linear-phase, Half-band f..."
2,\n Summary\n \n ...,10.1078/1434-8411-54100111,Asymptotic Analysis of Optimal Uniform Polar Q...,"[Peric, Zoran H., Stefanovic, Mihajlo C.]",2002-12-31,"[Optimal phase divisions, Optimal number of le..."
3,\n Summary\n \n ...,10.1078/1434-8411-54100114,Design and VHDL description of Multiplierless ...,"[Lutovac, Budimir M., Lutovac, Miroslav D.]",2002-12-31,[Digital signal processing]
4,\n Summary\n \n ...,10.1078/1434-8411-54100117,Application of Continuous Parameter Genetic Al...,"[Günel, Tayfun, Aydemir, Mustafa Emre]",2002-12-31,"[Bandpass amplifiers, Distributed amplifiers, ..."
5,,10.1078/1433-8319-00019,Editorial — recent developments in the journal,"[Kollmann, Johannes, Dietz, Hansjörg, Edwards,...",2002-12-31,[]
6,\n Abstract\n \n ...,10.1078/1433-8319-00020,Predicting distributions of species richness a...,"[Aarssen, Lonnie W., Schamp, Brandon S.]",2002-12-31,"[disturbance, diversity, habitat fertility, ‘l..."
7,\n Abstract\n \n ...,10.1078/1433-8319-00021,Ecological consequences and ontogeny of seed h...,"[Imbert, Eric]",2002-12-31,"[bet-hedging, dispersal, germination, plastici..."
8,\n Abstract\n \n ...,10.1078/1433-8319-00032,Shifts in dominance of native and invasive pla...,"[Woitke, Markus, Dietz, Hansjörg]",2002-12-31,"[clonal growth, disturbance, perennial forbs, ..."
9,\n Abstract\n \n ...,10.1078/1433-8319-00026,Estimations of the importance of plant resourc...,"[Stagegaard, Jesper, Sørensen, Marten, Kvist, ...",2002-12-31,"[forest/household surveys, forest resources, P..."


In [14]:
abstract_2001 = get_abstracts("date(2001)&sortBy=relevance", 1000, "Elsevier_abstracts - 2001")
abstract_2001

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,\n Summary\n \n ...,10.1078/1434-8411-00002,Engset Formulæ with Continuous Parameters – Th...,"[Iversen, Villy Bæk, Sanders, Bart]",2001-12-31,"[Generalised Engset formula, Blocking probabil..."
1,\n Summary\n \n ...,10.1078/1434-8411-00003,Routing and Wavelength Assignment in All-Optic...,"[Sun, Yong, Gu, Jun, Tsang, Danny H.K.]",2001-12-31,"[Routing and wavelength assignment, All-optica..."
2,\n Summary\n \n ...,10.1078/1434-8411-00006,Web Performance in Practice – Why We are Waiting,"[Charzinski, Joachim]",2001-12-31,"[Internet, WWW, QoS, HTTP, DNS, Loss, Latency]"
3,,10.1078/1434-8411-00023,German Radar Symposium GRS 2000 in Berlin,"[Rohling, Hermann]",2001-12-31,[]
4,\n Summary\n \n ...,10.1078/1434-8411-00011,A Quick Simulation Technique for a Fluid Infor...,"[Alcuri, Luigi, D'Acquisto, Giuseppe]",2001-12-31,"[Importance sampling, Markov-modulated fluid m..."
5,\n Summary\n \n ...,10.1078/1434-8411-00024,Passive Integration Methods: Fundamental Theory,"[Ochs, Karlheinz]",2001-12-31,"[Passive integration, Runge-Kutta methods, Wav..."
6,\n \n Almost ev...,10.1016/S0169-7161(01)19017-6,Stochastic processes in reliability,"[Kijima, Masaaki, Li, Haijun, Shaked, Moshe]",2001-12-31,[]
7,,10.1016/S0169-7161(01)20027-3,Ch. 25. System-based component test plans for ...,"[Rajgopal, Jayant, Mazumdar, Mainak]",2001-12-31,[]
8,,10.1016/S0169-7161(01)20028-5,Ch. 26. Life-test planning for preliminary scr...,"[Stein, Jeff, Doganaksoy, Necip]",2001-12-31,[]
9,\n Abstract\n \n ...,10.1078/1433-8319-00018,\n Cintractiella diplasiae – a s...,"[Piepenbring, Meike]",2001-12-31,[\n Cintractiella lamii\n ...


In [15]:
abstract_2000 = get_abstracts("date(2000)&sortBy=relevance", 1000, "Elsevier_abstracts - 2000")
abstract_2000

Artigos:  100
Artigos:  200
Artigos:  300
Artigos:  400
Artigos:  500
Artigos:  600
Artigos:  700
Artigos:  800
Artigos:  900
Artigos:  1000


,Abstract,DOI,Title,Creator,Date,Keywords
0,,10.1016/S0169-7161(00)18019-8,17 Measurement error models for environmental ...,"[Lyles, Robert H., Kupper, Lawrence L.]",2000-12-31,[]
1,,10.1016/S0169-7161(00)18020-4,18 Statistical perspectives in clinical epidem...,"[Bangdiwala, Shrikant I., Muñoz, Sergio R.]",2000-12-31,[]
2,,10.1016/S0169-7161(00)18001-0,Preface,"[Sen, Pranab K., C., Radhakrishna Rao]",2000-12-31,[]
3,,10.1016/S0169-7161(00)18011-3,9 Non-parametrics in bioenvironmental and publ...,"[Sen, Pranab Kumar]",2000-12-31,[]
4,,10.1016/S0169-7161(00)18026-5,24 Genomic sequences and quasi-multivariate CA...,"[Pinheiro, Hildete Prisco, Seillier-Moiseiwits...",2000-12-31,[]
5,,10.1016/S0169-7161(00)18013-7,11 Spatial statistical methods for environment...,"[Lawson, Andrew B., Cressie, Noel]",2000-12-31,[]
6,,10.1016/S1634-2143(00)72179-0,Colgajos cutáneos,"[Servant, J.M., Revol, M.]",2000-12-31,[]
7,\n Publisher Summary\n ...,10.1016/S0168-1273(00)30011-3,Chapter 194 Electronic 4f state splittings in ...,"[Staub, U., Soderholm, L.]",2000-12-31,[]
8,\n Publisher Summary\n ...,10.1016/S0168-1273(00)30006-X,Chapter 189 Single-crystal growth for science ...,"[Shiohara, Yuh, Goodilin, Eugene A.]",2000-12-31,[]
9,\n Publisher Summary\n ...,10.1016/S0168-1273(00)30005-8,Chapter 188 Crystal chemistry of superconducti...,"[Raveau, B., Michel, C., Hervieu, M.]",2000-12-31,[]
